In [75]:
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
import seaborn as sns
import bar_chart_race as bcr

from datetime import datetime
from pytz import timezone

In [2]:
df_1 = pd.read_json('/Users/Karolina/ProjectsDataScience/data_science_environment/data/MyDataSpotify/StreamingHistory1.json')
df_2 = pd.read_json('/Users/Karolina/ProjectsDataScience/data_science_environment/data/MyDataSpotify/StreamingHistory2.json')
df_3 = pd.read_json('/Users/Karolina/ProjectsDataScience/data_science_environment/data/MyDataSpotify/StreamingHistory3.json')
df_4 = pd.read_json('/Users/Karolina/ProjectsDataScience/data_science_environment/data/MyDataSpotify/StreamingHistory4.json')

In [5]:
data_dir = '/Users/Karolina/ProjectsDataScience/data_science_environment/data/'
file_list = ['MyDataSpotify/StreamingHistory1.json',
             'MyDataSpotify/StreamingHistory2.json',
             'MyDataSpotify/StreamingHistory3.json',
             'MyDataSpotify/StreamingHistory4.json']

dfs = [] # an empty list to store the data frames
for file in file_list:
    data = pd.read_json(data_dir+file) # read data frame from json file
    dfs.append(data) # append the data frame to the list

df = pd.concat(dfs, ignore_index=True) # concatenate all the data frames in the list.

In [6]:
df.head()

,endTime,artistName,trackName,msPlayed
0,2021-01-23 14:36,Billie Eilish,Lo Vas A Olvidar (with ROSALÍA),203464
1,2021-01-23 14:39,Billie Eilish,Lo Vas A Olvidar (with ROSALÍA),203464
2,2021-01-23 14:43,Billie Eilish,Lo Vas A Olvidar (with ROSALÍA),203464
3,2021-01-23 14:46,Billie Eilish,Lo Vas A Olvidar (with ROSALÍA),203464
4,2021-01-23 14:50,Billie Eilish,Lo Vas A Olvidar (with ROSALÍA),203464


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37735 entries, 0 to 37734
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   endTime     37735 non-null  object
 1   artistName  37735 non-null  object
 2   trackName   37735 non-null  object
 3   msPlayed    37735 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 1.2+ MB


In [118]:
#double checking for duplicates
df.duplicated().sum()

0

In [8]:
df.endTime = pd.to_datetime(df.endTime, infer_datetime_format=True) 

In [9]:
#Set the current time to US Pacific timezone
df['endTimeTz']= df.endTime.dt.tz_localize(tz='UTC')

#convert US Pacific to Europe/Berlin Timezone
df['endTimeTzEU']=df.endTimeTz.dt.tz_convert(tz='Europe/Berlin')


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37735 entries, 0 to 37734
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype                        
---  ------       --------------  -----                        
 0   endTime      37735 non-null  datetime64[ns]               
 1   artistName   37735 non-null  object                       
 2   trackName    37735 non-null  object                       
 3   msPlayed     37735 non-null  int64                        
 4   endTimeTz    37735 non-null  datetime64[ns, UTC]          
 5   endTimeTzEU  37735 non-null  datetime64[ns, Europe/Berlin]
 6   yearWeek     37735 non-null  object                       
dtypes: datetime64[ns, Europe/Berlin](1), datetime64[ns, UTC](1), datetime64[ns](1), int64(1), object(3)
memory usage: 2.0+ MB


In [114]:
weekly = df.groupby([pd.Grouper(key='endTime', freq='W'),'artistName'])['trackName'].size().reset_index()

weekly['no_csum'] = weekly.groupby(['artistName'])['trackName'].cumsum()

# weekly.loc[weekly['artistName'] == 'Ariana Grande']

#choosing only top5 artist in a given week
weekly5 = weekly.set_index(['endTime', 'artistName']).groupby(level=0, group_keys=False)['no_csum'].nlargest(5)

#reformatting the data into format suitable for the bar_chart_race package
weekly5_u = weekly5.unstack()
weekly5_u.fillna(method='ffill', inplace=True)
weekly5_u.fillna(0, inplace=True)

In [110]:
%%time
# help(bcr.bar_chart_race)
bcr.bar_chart_race(df=weekly5_u, n_bars=5, title='bla', period_length=400, cmap='dark12')

/opt/anaconda3/lib/python3.8/site-packages/bar_chart_race/_make_chart.py:286: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(self.df_values.columns)
/opt/anaconda3/lib/python3.8/site-packages/bar_chart_race/_make_chart.py:287: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([max_val] * len(ax.get_xticks()))
/opt/anaconda3/lib/python3.8/site-packages/bar_chart_race/_make_chart.py:226: UserWarning: Some of your columns never make an appearance in the animation. To reduce color repetition, set `filter_column_colors` to `True`
  warnings.warn("Some of your columns never make an appearance in the animation. "


CPU times: user 25.4 s, sys: 5.87 s, total: 31.3 s
Wall time: 40.4 s


In [105]:
weekly5_u

artistName,Ariana Grande,Biig Piig,Billie Eilish,Disclosure,Doja Cat,Drake,Frédéric Chopin,Giveon,Justin Bieber,Kanye West,Kehlani,Kygo,Leroy Sanchez,Ludwig van Beethoven,Olivia Rodrigo,Q,SZA,Taylor Swift,The Weeknd,Ty Dolla $ign
endTime,,,,,,,,,,,,,,,,,,,,
2021-01-24,0.0,4.0,73.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,0.0
2021-01-31,0.0,4.0,111.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,43.0,3.0,43.0,49.0,49.0,0.0,0.0,0.0
2021-02-07,207.0,4.0,135.0,0.0,0.0,0.0,5.0,0.0,79.0,0.0,0.0,0.0,43.0,3.0,43.0,50.0,62.0,0.0,0.0,0.0
2021-02-14,411.0,4.0,135.0,0.0,0.0,0.0,5.0,0.0,246.0,0.0,0.0,96.0,43.0,3.0,48.0,50.0,62.0,0.0,69.0,0.0
2021-02-21,420.0,4.0,137.0,0.0,0.0,0.0,5.0,0.0,395.0,0.0,0.0,107.0,43.0,3.0,48.0,50.0,62.0,0.0,69.0,209.0
2021-02-28,426.0,4.0,150.0,0.0,0.0,0.0,5.0,0.0,451.0,0.0,0.0,108.0,43.0,3.0,48.0,50.0,62.0,0.0,69.0,213.0
2021-03-07,440.0,4.0,310.0,236.0,0.0,0.0,5.0,0.0,564.0,0.0,0.0,108.0,43.0,3.0,48.0,50.0,62.0,0.0,69.0,215.0
2021-03-14,471.0,4.0,340.0,285.0,0.0,0.0,5.0,0.0,566.0,0.0,0.0,108.0,43.0,3.0,48.0,50.0,62.0,0.0,69.0,317.0
2021-03-21,636.0,4.0,350.0,292.0,0.0,0.0,5.0,0.0,1010.0,0.0,0.0,108.0,43.0,3.0,48.0,50.0,62.0,0.0,69.0,323.0


In [116]:
weekly_time = df.groupby([pd.Grouper(key='endTime', freq='W'),'artistName'])['msPlayed'].sum().reset_index()

weekly_time['no_csum'] = weekly_time.groupby(['artistName'])['msPlayed'].cumsum()

# weekly.loc[weekly['artistName'] == 'Ariana Grande']

#choosing only top5 artist in a given week
weekly_time5 = weekly_time.set_index(['endTime', 'artistName']).groupby(level=0, group_keys=False)['no_csum'].nlargest(5)

#reformatting the data into format suitable for the bar_chart_race package
weekly_time5_u = weekly_time5.unstack()
weekly_time5_u.fillna(method='ffill', inplace=True)
weekly_time5_u.fillna(0, inplace=True)

In [117]:
%%time
# help(bcr.bar_chart_race)
bcr.bar_chart_race(df=weekly_time5_u, n_bars=5, title='bla', period_length=400, cmap='dark12')

/opt/anaconda3/lib/python3.8/site-packages/bar_chart_race/_make_chart.py:286: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(self.df_values.columns)
/opt/anaconda3/lib/python3.8/site-packages/bar_chart_race/_make_chart.py:287: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([max_val] * len(ax.get_xticks()))
/opt/anaconda3/lib/python3.8/site-packages/bar_chart_race/_make_chart.py:226: UserWarning: Some of your columns never make an appearance in the animation. To reduce color repetition, set `filter_column_colors` to `True`
  warnings.warn("Some of your columns never make an appearance in the animation. "


CPU times: user 22.6 s, sys: 5.1 s, total: 27.7 s
Wall time: 31.7 s
